<a href="https://colab.research.google.com/github/cdefaz/MSCI546/blob/main/logistic_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

import sklearn
from sklearn.linear_model import LogisticRegression

import sklearn.metrics as sm
from sklearn import datasets
from sklearn.metrics import confusion_matrix, classification_report

from sklearn.model_selection import KFold
from sklearn.metrics import \
    silhouette_score, adjusted_rand_score, accuracy_score, precision_score, recall_score, f1_score

In [ ]:
import pandas as pd

# Kaggle competition: https://www.kaggle.com/competitions/digit-recognizer/data

def load_simple(file: str):
    df = pd.read_csv(file)
    label = df['label'].copy()
    df[df < 85] = -1
    df[(df >= 85) & (df <= 170)] = 0
    df[df > 170] = 1
    df['label'] = label
    return df

In [ ]:
train_df_simplified = load_simple('/content/digits-train.csv')
train_df_simplified

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
1,0,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
2,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
3,4,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
4,0,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41995,0,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
41996,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
41997,7,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
41998,6,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1


In [ ]:
train_X = train_df_simplified.iloc[:, 1:785]
train_X

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
2,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
3,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
4,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41995,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
41996,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
41997,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
41998,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1


In [ ]:
train_y = train_df_simplified.iloc[:, 0]
train_y

0        1
1        0
2        1
3        4
4        0
        ..
41995    0
41996    1
41997    7
41998    6
41999    9
Name: label, Length: 42000, dtype: int64

In [ ]:
# Define logistic regression model
def define_model():
  model = LogisticRegression(verbose=2)
  return model

In [ ]:
# Referenced https://medium.com/@muhammetbolat/supervised-unsupervised-techniques-on-mnist-dataset-3f2ffd4c41c5
# Referenced Tutorial 5

# Evaluate a model using k-fold cross-validation
def evaluate_model(dataX, dataY, n_folds):
	metrics = {
			'Accuracy': [],
			'Precision': [],
			'Recall': [],
			'F1 Score': []
	}
	scores = pd.DataFrame(metrics)
	# Prepare cross validation
	kfold = KFold(n_folds, shuffle=True, random_state=1)
	# Enumerate splits
	for train_ix, test_ix in kfold.split(dataX):
		# Call defined logistic regression model
		lr = define_model()
		# Select rows for train and test
		trainX, trainY, testX, testY = dataX.iloc[train_ix], dataY.iloc[train_ix], dataX.iloc[test_ix], dataY.iloc[test_ix]
		# Fit model
		lr.fit(trainX, trainY)
		number_labels = lr.predict(testX)
		true_labels = testY

		# For each fold, calculate metrics (to be averaged together at the end)
		result = {
			'Accuracy': round(accuracy_score(true_labels, number_labels), 4) * 100,
			'Precision': round(precision_score(true_labels, number_labels, average = 'macro'), 4) * 100,
			'Recall': round(recall_score(true_labels, number_labels, average = 'macro'), 4) * 100,
			'F1 Score': round(f1_score(true_labels, number_labels, average = 'macro'), 4) * 100
		}
		scores = scores.append(result, ignore_index=True)
		# Return scores for each fold
	return scores

In [ ]:
# Define number of folds for logisitic regression model
folds = 3
lr = evaluate_model(train_X, train_y, folds)

# Print returned scores for each fold
print("\n")
print(lr)

# Calculate the average of each metric across all folds
avg_accuracy = lr['Accuracy'].mean()
avg_precision = lr['Precision'].mean()
avg_recall = lr['Recall'].mean()
avg_accuracy = lr['F1 Score'].mean()

# Print averages
print("\n")
print("Average Accuracy: ", avg_accuracy)
print("Average Precision: ", avg_precision)
print("Average Recall: ", avg_recall)
print("Average Accuracy: ", avg_accuracy)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
<ipython-input-61-726cb7976b85>:52: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  scores = scores.append(result, ignore_index=True)
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https:



   Accuracy  Precision  Recall  F1 Score
0     91.76      91.65   91.66     91.65
1     90.89      90.77   90.73     90.74
2     91.09      90.98   91.00     90.99


Average Accuracy:  91.12666666666667
Average Precision:  91.13333333333333
Average Recall:  91.13
Average Accuracy:  91.12666666666667


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
<ipython-input-61-726cb7976b85>:52: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  scores = scores.append(result, ignore_index=True)
